efficientformer 모델 돌리기

파이토치 사용
이진분류용 모델
가중치 저장

0. CUDA 버전 확인 후, 여기에 맞는 PyTorch 설치 진행
nvidia-smi | CUDA Version: 12.7

-> 파이토치는 쿠다 12.1까지 지원하므로 cu121로 설치진행

1. PyTorch 설치 명령어 

pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

2. git clone https://github.com/snap-research/EfficientFormer.git

3. pip install timm==0.6.13 einops




In [1]:
# 설치 완료 후 버전 확인
import torch, timm

print("PyTorch:", torch.__version__)
print("CUDA:", torch.version.cuda)
print("Current device:", torch.cuda.current_device())
print("GPU name:", torch.cuda.get_device_name(0))
print("GPU", torch.cuda.is_available())
print("timm", timm.__version__)

PyTorch: 2.9.1+cu128
CUDA: 12.8
Current device: 0
GPU name: NVIDIA A40
GPU True
timm 0.6.13


In [12]:
#EfficientFormer 로딩

import sys
sys.path.append("/workspace/EfficientFormer")


In [13]:
# 이진분류 모델 설정
from models.efficientformer import efficientformer_l1
import torch.nn as nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # gpu 못쓰면 cpu로 자동변경
model = efficientformer_l1(pretrained=False)
model.classifier = nn.Linear(512, 2)    # 이진분류로 변경
                                        # input → backbone → classifier(1000) 기본구조
                                        # input → backbone → classifier(2)  이걸 FC 출력 2로 바꿈


model = model.to(device)

print("Model loaded on:", next(model.parameters()).device) # 모델 로딩 확인


Model loaded on: cuda:0


In [15]:
from torchvision import transforms, datasets
from torch.utils.data import DataLoader

train_dir = "/workspace/merge_data/data/img/train"
val_dir   = "/workspace/merge_data/data/img/val"

# 이미지 전처리
transform = transforms.Compose([
    transforms.Resize((224, 224)), #L1~L7까지 224동일
    transforms.ToTensor(),
])

# 커스텀 이진 레이블 데이터셋
class HappyDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.img_paths = []
        self.labels = []

        # root_dir 하위 폴더들을 자동 탐색
        for emo in os.listdir(root_dir):
            emo_dir = os.path.join(root_dir, emo)
            if not os.path.isdir(emo_dir):
                continue

            for file in os.listdir(emo_dir):
                if not file.lower().endswith(("jpg", "jpeg", "png")):
                    continue
                
                img_path = os.path.join(emo_dir, file)

                # 깨진 이미지 미리 필터링
                img = cv2.imread(img_path)
                if img is None:
                    continue

                self.img_paths.append(img_path)
                # happy = 1, 나머지 모두 0
                self.labels.append(1 if emo == "happy" else 0)

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        img_path = self.img_paths[idx]
        label = self.labels[idx]

        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        if self.transform:
            img = self.transform(img)

        return img, label


train_dataset = HappyDataset(train_dir, transform=transform)
val_dataset   = HappyDataset(val_dir, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
val_loader   = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)

len(train_dataset), len(val_dataset)


Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG


KeyboardInterrupt: 

In [ ]:
import torch.optim as optim

#손실함수 및 옵티마이저
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)


In [ ]:
from tqdm import tqdm

# train / val 베이스라인 모델
def train_one_epoch(model, loader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    for imgs, labels in tqdm(loader, desc="Train"):
        imgs, labels = imgs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(imgs)

        # EfficientFormer는 (logits, extra_features) 형태로 tuple 반환할 수 있음 → 첫 번째만 사용
        if isinstance(outputs, tuple):
            outputs = outputs[0]

        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, preds = outputs.max(1)
        correct += preds.eq(labels).sum().item()
        total += labels.size(0)

    return total_loss / len(loader), correct / total


def validate(model, loader, criterion, device):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0

    with torch.no_grad(): #inffernet로 바꿔주세요 요청받음
        for imgs, labels in tqdm(loader, desc="Validation"):
            imgs, labels = imgs.to(device), labels.to(device)

            outputs = model(imgs)

            if isinstance(outputs, tuple):
                outputs = outputs[0]

            loss = criterion(outputs, labels)

            total_loss += loss.item()
            _, preds = outputs.max(1)
            correct += preds.eq(labels).sum().item()
            total += labels.size(0)

    return total_loss / len(loader), correct / total


In [ ]:
import copy
# 학습루프 + 최적가중치 저장
EPOCHS = 50
best_acc = 0
best_wts = copy.deepcopy(model.state_dict())

for epoch in range(EPOCHS):
    print(f"\n🔵 Epoch {epoch+1}/{EPOCHS}")

    train_loss, train_acc = train_one_epoch(model, train_loader, optimizer, criterion, device)
    val_loss, val_acc     = validate(model, val_loader, criterion, device)

    print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}")
    print(f"Val   Loss: {val_loss:.4f} | Val   Acc: {val_acc:.4f}")

    if val_acc > best_acc:
        best_acc = val_acc
        best_wts = copy.deepcopy(model.state_dict())
        torch.save(best_wts, "efficientformer_binary_best.pth")
        print("💾 Best model saved!")



🔵 Epoch 1/50


Train:   0%|          | 0/183 [00:00<?, ?it/s]

Validation: 100%|██████████| 36/36 [00:28<00:00,  1.27it/s]


Train Loss: 1.9074 | Train Acc: 0.6923
Val   Loss: 1.6227 | Val   Acc: 0.7402
💾 Best model saved!

🔵 Epoch 2/50


Validation: 100%|██████████| 36/36 [00:27<00:00,  1.31it/s]


...

Train Loss: 0.0276 | Train Acc: 0.9892
Val   Loss: 1.2530 | Val   Acc: 0.6516

🔵 Epoch 49/50


Validation: 100%|██████████| 36/36 [00:28<00:00,  1.25it/s]


Train Loss: 0.0108 | Train Acc: 0.9976
Val   Loss: 1.3594 | Val   Acc: 0.6447

🔵 Epoch 50/50


Validation: 100%|██████████| 36/36 [00:28<00:00,  1.28it/s]

Train Loss: 0.0150 | Train Acc: 0.9952
Val   Loss: 1.3759 | Val   Acc: 0.6507


In [ ]:
# 여기서부터 efficient_former GPU 최적화 버전

In [ ]:
#pip install albumentations==1.3.0 opencv-python
#pip install timm

In [ ]:
import sys, os
sys.stderr = open(os.devnull, "w")     # stderr 완전 무음 처리
os.environ["OPENCV_LOG_LEVEL"] = "SILENT"


In [3]:
# ============================================================
# 0. 기본 환경 설정
# ============================================================
import os
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


import os
import cv2
import torch
from torch.utils.data import Dataset
from PIL import Image
import numpy as np



def safe_load_image(path):
    """고속 로딩 + 깨진 JPEG 자동 처리"""
    try:
        img = cv2.imread(path)
        if img is None:
            # cv2가 못읽으면 Pillow로 시도
            img = Image.open(path).convert("RGB")
            img = np.array(img)
        else:
            # BGR → RGB
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        return img
    except Exception:
        return None


class FastHappyDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.paths = []
        self.labels = []
        self.transform = transform

        for emo in os.listdir(root_dir):
            emo_dir = os.path.join(root_dir, emo)
            if not os.path.isdir(emo_dir):
                continue

            label = 1 if emo == "happy" else 0

            for f in os.listdir(emo_dir):
                if f.lower().endswith(("jpg", "jpeg", "png")):
                    self.paths.append(os.path.join(emo_dir, f))
                    self.labels.append(label)

        print(f"Loaded {len(self.paths)} images from {root_dir}")

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        path = self.paths[idx]
        label = self.labels[idx]

        img = safe_load_image(path)

        # 완전 깨진 이미지 제거
        if img is None:
            return self.__getitem__((idx + 1) % len(self.paths))

        if self.transform:
            img = self.transform(image=img)["image"]

        return img, torch.tensor(label).long()


Using device: cuda


In [4]:
# ============================================================
# 1. Albumentations Transform
# ============================================================
import albumentations as A
from albumentations.pytorch import ToTensorV2

transform = A.Compose([
    A.Resize(224, 224),
    A.HorizontalFlip(p=0.5),
    A.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ),
    ToTensorV2(),
])


# ============================================================
# 2. Dataset & DataLoader
# ============================================================
train_root = "/workspace/merge_data/data/img/train"
val_root   = "/workspace/merge_data/data/img/val"

train_dataset = FastHappyDataset(train_root, transform=transform)
val_dataset   = FastHappyDataset(val_root, transform=transform)

train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=32,
    shuffle=True,
    num_workers=8,
    pin_memory=True,
    persistent_workers=True,
    prefetch_factor=4,
)

val_loader = torch.utils.data.DataLoader(
    val_dataset,
    batch_size=32,
    shuffle=False,
    num_workers=8,
    pin_memory=True,
    persistent_workers=True,
    prefetch_factor=4,
)


# ============================================================
# 3. EfficientFormer 모델 불러오기
# ============================================================
import sys
sys.path.append("/workspace/EfficientFormer")

from models.efficientformer import efficientformer_l1

model = efficientformer_l1(pretrained=True)
model.classifier = nn.Linear(512, 2)   # binary classification

model = model.to(device)
print("Model loaded on:", next(model.parameters()).device)


# ============================================================
# 4. Optimizer & Loss
# ============================================================
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=3e-5, weight_decay=1e-5)

scaler = torch.amp.GradScaler("cuda")


# ============================================================
# 5. 학습 함수
# ============================================================
def train_one_epoch():
    model.train()
    total_loss, total_correct, total_samples = 0, 0, 0

    for imgs, labels in tqdm(train_loader, desc="Train"):
        imgs = imgs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        with torch.amp.autocast("cuda"):
            outputs = model(imgs)
            if isinstance(outputs, tuple):
                outputs = outputs[0]
            loss = criterion(outputs, labels)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        preds = outputs.argmax(dim=1)
        total_correct += (preds == labels).sum().item()
        total_loss += loss.item() * labels.size(0)
        total_samples += labels.size(0)

    return total_loss / total_samples, total_correct / total_samples


# ============================================================
# 6. 검증 함수
# ============================================================
def validate():
    model.eval()
    total_loss, total_correct, total_samples = 0, 0, 0

    with torch.inference_mode(), torch.amp.autocast("cuda"):
        for imgs, labels in tqdm(val_loader, desc="Validation"):
            imgs = imgs.to(device)
            labels = labels.to(device)

            outputs = model(imgs)
            if isinstance(outputs, tuple):
                outputs = outputs[0]

            loss = criterion(outputs, labels)
            preds = outputs.argmax(dim=1)

            total_correct += (preds == labels).sum().item()
            total_loss += loss.item() * labels.size(0)
            total_samples += labels.size(0)

    return total_loss / total_samples, total_correct / total_samples


# ============================================================
# 7. 전체 학습 루프
# ============================================================
num_epochs = 10
best_acc = 0

for epoch in range(num_epochs):
    print(f"\n===== Epoch {epoch+1}/{num_epochs} =====")

    train_loss, train_acc = train_one_epoch()
    val_loss, val_acc     = validate()

    print(f"Train Loss: {train_loss:.4f} | Acc: {train_acc:.4f}")
    print(f"Val   Loss: {val_loss:.4f} | Acc: {val_acc:.4f}")

    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), "efficientformer_fastloader_best.pth")
        print("💾 Best model saved!")

print(f"\n🎉 Training Completed — Best Val Acc: {best_acc:.4f}")

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt

model.eval()
all_preds, all_labels = [], []

with torch.inference_mode(), torch.amp.autocast("cuda"):
    for imgs, labels in tqdm(val_loader, desc="Final Evaluation"):
        imgs, labels = imgs.to(device), labels.to(device)
        outputs = model(imgs)
        if isinstance(outputs, tuple):
            outputs = outputs[0]
        preds = outputs.argmax(1)

        all_preds.extend(preds.cpu().tolist())
        all_labels.extend(labels.cpu().tolist())

acc  = accuracy_score(all_labels, all_preds)
prec = precision_score(all_labels, all_preds, zero_division=0)
rec  = recall_score(all_labels, all_preds, zero_division=0)
f1   = f1_score(all_labels, all_preds, zero_division=0)

print("\n===== Final Metrics =====")
print(f"Accuracy:  {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"Recall:    {rec:.4f}")
print(f"F1 Score:  {f1:.4f}")
print("\nClassification Report:")
print(classification_report(all_labels, all_preds, digits=4))

cm = confusion_matrix(all_labels, all_preds)
plt.figure(figsize=(5,4))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=["Not Happy (0)", "Happy (1)"],
            yticklabels=["Not Happy (0)", "Happy (1)"])
plt.title("Confusion Matrix")
plt.show()


Loaded 5841 images from /workspace/merge_data/data/img/train
Loaded 1151 images from /workspace/merge_data/data/img/val
Model loaded on: cuda:0

===== Epoch 1/10 =====
Train Loss: 4.4740 | Acc: 0.5535
Val   Loss: 3.4003 | Acc: 0.7194
💾 Best model saved!

===== Epoch 2/10 =====


Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS

KeyboardInterrupt: 